In [24]:
def is_consecutive_neighbors_same_room(floorplan, cell, direction):
    directions = {'north': [(-1,0), (-1,-1), (-1,1)],
                       'south': [(1,0), (1,-1), (1,1)],
                       'west' : [(0,-1), (-1,-1), (1, -1)],
                       'east': [(0,1), (-1,1), (1,1)]
                }
    consecutive3 = []
    rows, cols = floorplan.shape
    for d in directions[direction]:
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        # 경계를 벗어났는지 확인
        if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
            consecutive3.append(neighbor)
    
    if len(consecutive3) < 3:
        return False
    is_consecutive = all([floorplan[n] == floorplan[cell] for n in consecutive3]) # 현재셀과 세 개의 연속셀이 모두 같으면
    # print(f'[is_consecutive_neighbors_same_room]{cell} to  is_consecutive = {is_consecutive}')
    return is_consecutive

def consecutive_neighbor_in_orientation(floorplan, cell):
    orientation = ['north', 'south', 'west', 'east']
    count_true = 0
    triple_match_directions = []
    for o in orientation:
        if is_consecutive_neighbors_same_room(floorplan, cell, o):
            triple_match_directions.append(o)
    # print(f'[{cell}]: triple_match_directions{triple_match_directions}')
    return triple_match_directions
        
def is_cascading_corner(triple_match_directions): # consecutive3_direction 세 셀이 같은 방향
    if len(triple_match_directions) != 2: return False
    if 'north' in triple_match_directions  or 'south' in triple_match_directions :
        if 'east'  in triple_match_directions or 'west' in triple_match_directions :
            return True
    if 'east' in  triple_match_directions or  'west' in triple_match_directions:
        if ('north' in triple_match_directions  or 'south' in triple_match_directions) :
            return True

def all_neighbors(cell, floorplan):
    directions4 = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    neighbors = []
    for d in directions4:
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        if 0 <= neighbor[0] < floorplan.shape[0] and 0 <= neighbor[1] < floorplan.shape[1] and floorplan[
            neighbor] > 0:
            neighbors.append(neighbor)
    return neighbors

def get_diff_ratio_neighbors(floorplan, cell):
    # cells = all_active_neighbors8(cell, floorplan)
    cells = all_neighbors(cell, floorplan)
    diff_ratio_dict = {}
    for neighbor_cell in cells:
        if floorplan[cell] != floorplan[neighbor_cell]:
            diff_ratio = get_diff_ratio_cell(floorplan, neighbor_cell)
            diff_ratio_dict[neighbor_cell] = diff_ratio
    return diff_ratio_dict

def get_diff_ratio_cell(floorplan, cell):
    # neighbors = all_active_neighbors8(cell, floorplan)
    # neighbors = all_active_neighbors(cell, floorplan)
    neighbors = all_neighbors(cell, floorplan)
    max_neighbors = len(neighbors)
    if max_neighbors == 0. :
        print(neighbors, cell)
    num_diff_neighbors = sum(
        1 for neighbor in neighbors if floorplan[neighbor[0], neighbor[1]] != floorplan[cell[0], cell[1]])
    diff_ratio = num_diff_neighbors / max_neighbors
    # print(f'max_neighbors = {max_neighbors}, num_diff_neighbors = {num_diff_neighbors}, diff_ratio = {diff_ratio} returning {diff_ratio}')
    # print(f'\t\t\t[get_diff_ratio_cell] ...when {cell}={floorplan[cell]}s diff_ratio = {diff_ratio} max_neighbors {max_neighbors}')
    return diff_ratio

def create_diff_ratio_array(floorplan):
    diff_ratios = np.zeros_like(floorplan, dtype=float)
    
    for i in range(floorplan.shape[0]):
        for j in range(floorplan.shape[1]):
            cell = (i,j)            
            if is_valid_cell(floorplan, cell):
                diff_ratios[cell] = get_diff_ratio_cell(floorplan, cell)
                print(f'{cell} diff_ratio={diff_ratios[cell]}')
    return diff_ratios

def is_valid_cell(floorplan, cell):
    is_valid = 0 <= cell[0] < floorplan.shape[0] and 0 <= cell[1] < floorplan.shape[1] and floorplan[cell] > 0
    # print(f'{cell} is_valid={is_valid}')
    return is_valid
        
import numpy as np
floorplan = np.array([
        [5, 4, 4, 4, 4, 3, 3, 3, 3, 2, -1, -1, -1, -1],
        [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, -1, -1, -1, -1],
        [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1],
        [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1],
        [5, 4, 4, 4, 3, 3, 3, 2, 2, 1, 1, 1, -1, -1],
        [5, 5, 5, 4, 3, 3, 3, 1, 1, 1, 1, 1, -1, -1],
        [-1,-1,-1,-1,3, 3, 1, 1, -1, -1, -1, -1, -1, -1],
        [-1,-1,-1,-1,3, 1, 1, 1, -1, -1, -1, -1, -1, -1]
    ])
np.set_printoptions(precision=1, suppress=True)
diff_ratios = create_diff_ratio_array(floorplan)
print(f'diff_ratios=\n{diff_ratios}')
aligned_corners = np.zeros_like(floorplan)
directions = ['west', 'east', 'south', 'north']
for i in range(floorplan.shape[0]):
    for j in range(floorplan.shape[1]):
        cell = (i,j)
        if is_valid_cell(floorplan, cell) :
        # for direction in directions:
        #     aligned3 = is_consecutive_neighbors_same_room(floorplan, cell, direction)
        #     print(f'{cell}:{direction} = {aligned3}')
            
            triple_match_directions = consecutive_neighbor_in_orientation(floorplan, cell)
            # print(f'triple_match_directions {cell} = {triple_match_directions}')
            aligned_corners[i, j] = is_cascading_corner( triple_match_directions)
print(f'aligned_corners=\n{aligned_corners}')
mask = (diff_ratios > 0.5) | (( diff_ratios == 0.5) & (aligned_corners == 1))
print(mask.astype(int))

# for idx in np.ndindex(floorplan.shape):
#     print(f"{idx}: {floorplan[idx]}")

# consecutive_neighbor_in_orientation(floorplan, cell)

(0, 0) diff_ratio=0.5
(0, 1) diff_ratio=0.3333333333333333
(0, 2) diff_ratio=0.0
(0, 3) diff_ratio=0.0
(0, 4) diff_ratio=0.6666666666666666
(0, 5) diff_ratio=0.3333333333333333
(0, 6) diff_ratio=0.0
(0, 7) diff_ratio=0.3333333333333333
(0, 8) diff_ratio=0.6666666666666666
(0, 9) diff_ratio=0.5
(1, 0) diff_ratio=0.3333333333333333
(1, 1) diff_ratio=0.25
(1, 2) diff_ratio=0.0
(1, 3) diff_ratio=0.25
(1, 4) diff_ratio=0.5
(1, 5) diff_ratio=0.0
(1, 6) diff_ratio=0.25
(1, 7) diff_ratio=0.5
(1, 8) diff_ratio=0.25
(1, 9) diff_ratio=0.0
(2, 0) diff_ratio=0.3333333333333333
(2, 1) diff_ratio=0.25
(2, 2) diff_ratio=0.0
(2, 3) diff_ratio=0.25
(2, 4) diff_ratio=0.25
(2, 5) diff_ratio=0.0
(2, 6) diff_ratio=0.25
(2, 7) diff_ratio=0.25
(2, 8) diff_ratio=0.0
(2, 9) diff_ratio=0.0
(2, 10) diff_ratio=0.3333333333333333
(2, 11) diff_ratio=0.6666666666666666
(2, 12) diff_ratio=0.3333333333333333
(2, 13) diff_ratio=0.0
(3, 0) diff_ratio=0.3333333333333333
(3, 1) diff_ratio=0.25
(3, 2) diff_ratio=0.0
(3, 3) 

In [32]:
import numpy as np

def is_corner_room(floorplan, cell):

    # 경계 검사:
    # 셀이 배열의 경계에 있는지 확인합니다. 셀이 경계에 있으면 True를 반환합니다.
    # -1 검사:
    # 이웃 셀들 중 -1이 있는지 확인합니다. 이웃 셀 중 하나라도 -1이면 True를 반환합니다.
    # 이웃 셀 확인:
    # 이웃 셀들 중 floorplan[cell]과 같은 값을 가지는 셀들의 위치를 저장합니다.
    # 코너 셀 조건:
    # 인접한 방이 1개 또는 2개인 경우 코너 셀로 간주하여 True를 반환합니다.

    rows, cols = floorplan.shape
    room_value = floorplan[cell]
    
    # 방향: 상, 하, 좌, 우, 좌상, 우상, 좌하, 우하
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    
    # 셀이 유효한 범위에 있는지 확인하는 함수
    def is_valid(cell):
        return 0 <= cell[0] < rows and 0 <= cell[1] < cols and room_value > 0
    
    # 경계에 있는 경우 코너로 간주
    if cell[0] == 0 or cell[0] == rows - 1 or cell[1] == 0 or cell[1] == cols - 1:
        return True
    
    # 이웃 셀들 중 -1이 있는지 확인
    for d in directions:
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        if is_valid(neighbor) and floorplan[neighbor] == -1:
            return True
    
    # 이웃 셀들 중 floorplan[cell]과 같은 값을 가지는 셀들의 위치를 저장
    neighbor_same_room = [cell for d in directions if is_valid((cell[0] + d[0], cell[1] + d[1])) and floorplan[(cell[0] + d[0], cell[1] + d[1])] == room_value]

    # 코너 셀은 인접한 방이 1개 또는 2개인 경우 (대각선 방향이 방의 내부에 포함되지 않는 경우)
    if len(neighbor_same_room) <= 2:
        return True
    
    return False

# 테스트 배열
floorplan = np.array([
    [5, 4, 4, 4, 4, 3, 3, 3, 3, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 1, 1, 1, -1, -1],
    [5, 5, 5, 4, 3, 3, 3, 1, 1, 1, 1, 1, -1, -1],
    [-1, -1, -1, -1, 3, 3, 1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 3, 1, 1, 1, -1, -1, -1, -1, -1, -1]
])

# 코너 셀 여부를 저장할 배열 초기화
corner_cells = np.zeros_like(floorplan, dtype=bool)

# 모든 셀에 대해 코너 셀 여부를 판단
for i in range(floorplan.shape[0]):
    for j in range(floorplan.shape[1]):
        corner_cells[i, j] = is_corner_room(floorplan, (i, j))
print(corner_cells.astype(int))
# 테스트 실행
test_cells = [(0, 0), (5,0),(0,4),(0,5), (0,8), (0,9), (1,4), (1,9),(2,11), (2,12),(2,13), (3,9), (3,10), (3,11), (3,13), (4,8), (4,9), (5,2), (5,3), (5,6), (5,7),(5,11),
              (6,5), (6,6),(7,4),(7,5), (1, 1), (2, 5), (5, 5), (4, 4)]
results = {cell: is_corner_room(floorplan, cell) for cell in test_cells}
print(results)


[[1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
{(0, 0): True, (5, 0): True, (0, 4): True, (0, 5): True, (0, 8): True, (0, 9): True, (1, 4): False, (1, 9): True, (2, 11): True, (2, 12): True, (2, 13): True, (3, 9): False, (3, 10): False, (3, 11): True, (3, 13): True, (4, 8): False, (4, 9): False, (5, 2): True, (5, 3): True, (5, 6): False, (5, 7): True, (5, 11): True, (6, 5): False, (6, 6): False, (7, 4): True, (7, 5): True, (1, 1): False, (2, 5): False, (5, 5): False, (4, 4): False}


In [31]:
import numpy as np

def is_corner_room(floorplan, cell):
    rows, cols = floorplan.shape
    room_value = floorplan[cell]
    
    # 방향: 상, 하, 좌, 우, 좌상, 우상, 좌하, 우하
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    
    # 셀이 유효한 범위에 있는지 확인하는 함수
    def is_valid(cell):
        return 0 <= cell[0] < rows and 0 <= cell[1] < cols
    
    # 경계에 있는 경우 코너로 간주
    if cell[0] == 0 or cell[0] == rows - 1 or cell[1] == 0 or cell[1] == cols - 1:
        return True
    
    # 이웃 셀들 중 -1이 있는지 확인
    for d in directions:
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        if is_valid(neighbor) and floorplan[neighbor] == -1:
            return True
    
    # 이웃 셀들 중 floorplan[cell]과 같은 값을 가지는 셀들의 위치를 저장
    neighbor_same_room = [cell for d in directions if is_valid((cell[0] + d[0], cell[1] + d[1])) and floorplan[(cell[0] + d[0], cell[1] + d[1])] == room_value]

    # 코너 셀은 인접한 방이 1개 또는 2개인 경우 (대각선 방향이 방의 내부에 포함되지 않는 경우)
    if len(neighbor_same_room) <= 2:
        return True
    
    return False

# 테스트 배열
floorplan = np.array([
    [5, 4, 4, 4, 4, 3, 3, 3, 3, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 1, 1, 1, -1, -1],
    [5, 5, 5, 4, 3, 3, 3, 1, 1, 1, 1, 1, -1, -1],
    [-1, -1, -1, -1, 3, 3, 1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 3, 1, 1, 1, -1, -1, -1, -1, -1, -1]
])

# 코너 셀 여부를 저장할 배열 초기화
corner_cells = np.zeros_like(floorplan, dtype=bool)

# 모든 셀에 대해 코너 셀 여부를 판단
for i in range(floorplan.shape[0]):
    for j in range(floorplan.shape[1]):
        corner_cells[i, j] = is_corner_room(floorplan, (i, j))

# 결과 출력
print(corner_cells.astype(int))


[[1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


### floorplan의 각 셀에 대해 경계 셀인지 여부를 True/False로 표시
#### is_boundary_cell 함수:
주어진 셀이 경계 셀인지 여부를 판단합니다.
주어진 셀의 이웃 셀이 경계 바깥에 있거나 -1인 경우 True를 반환합니다.
#### boundary_cells 배열 초기화:
floorplan과 같은 크기의 boundary_cells 배열을 초기화합니다. 이 배열은 각 셀이 경계 셀인지 여부를 True/False로 저장합니다.
모든 셀에 대해 경계 셀 여부 판단:
이중 for 루프를 사용하여 floorplan의 모든 셀을 순회하며 is_boundary_cell 함수를 적용하고, 그 결과를 boundary_cells 배열에 저장합니다

In [34]:
import numpy as np

def is_boundary_cell(floorplan, cell):
    rows, cols = floorplan.shape
    
    # 방향: 상, 하, 좌, 우, 좌상, 우상, 좌하, 우하
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    
    # 셀이 유효한 범위에 있는지 확인하는 함수
    def is_valid(cell):
        return 0 <= cell[0] < rows and 0 <= cell[1] < cols
    
    # 이웃 셀들 중 경계 바깥에 있거나 -1이 있는지 확인
    for d in directions:
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        if not is_valid(neighbor) or floorplan[neighbor] == -1:
            return True
    
    return False

# 테스트 배열
floorplan = np.array([
    [5, 4, 4, 4, 4, 3, 3, 3, 3, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 1, 1, 1, -1, -1],
    [5, 5, 5, 4, 3, 3, 3, 1, 1, 1, 1, 1, -1, -1],
    [-1, -1, -1, -1, 3, 3, 1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 3, 1, 1, 1, -1, -1, -1, -1, -1, -1]
])

# 경계 셀 여부를 저장할 배열 초기화
boundary_cells = np.zeros_like(floorplan, dtype=bool)

# 모든 셀에 대해 경계 셀 여부를 판단
for i in range(floorplan.shape[0]):
    for j in range(floorplan.shape[1]):
        boundary_cells[i, j] = is_boundary_cell(floorplan, (i, j))

# 결과 출력
print(boundary_cells.astype(int))


[[1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


헷갈리니까 원래 -1인 셀은 그냥 -1로 표시,

In [38]:
import numpy as np

def is_boundary_cell(floorplan, cell):
    rows, cols = floorplan.shape
    
    # 방향: 상, 하, 좌, 우, 좌상, 우상, 좌하, 우하
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    
    # 셀이 유효한 범위에 있는지 확인하는 함수
    def is_valid(cell):
        return 0 <= cell[0] < rows and 0 <= cell[1] < cols
    
    # 이웃 셀들 중 경계 바깥에 있거나 -1이 있는지 확인
    for d in directions:
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        if not is_valid(neighbor) or floorplan[neighbor] == -1:
            return True
    
    return False

# 테스트 배열
floorplan = np.array([
    [5, 4, 4, 4, 4, 3, 3, 3, 3, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 1, 1, 1, -1, -1],
    [5, 5, 5, 4, 3, 3, 3, 1, 1, 1, 1, 1, -1, -1],
    [-1, -1, -1, -1, 3, 3, 1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 3, 1, 1, 1, -1, -1, -1, -1, -1, -1]
])

# 경계 셀 여부를 저장할 배열 초기화
boundary_cells = np.full(floorplan.shape, False, dtype=int)

# 모든 셀에 대해 경계 셀 여부를 판단
for i in range(floorplan.shape[0]):
    for j in range(floorplan.shape[1]):
        if floorplan[i, j] == -1:
            boundary_cells[i, j] = -1
        else:
            boundary_cells[i, j] = is_boundary_cell(floorplan, (i, j))

# 결과 출력
for row in boundary_cells:
    print("[", " ".join(f"{int(x):2}" if x != -1 else f"{x:2}" for x in row), "]")


[  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 ]
[  1  0  0  0  0  0  0  0  0  1 -1 -1 -1 -1 ]
[  1  0  0  0  0  0  0  0  0  1  1  1  1  1 ]
[  1  0  0  0  0  0  0  0  0  0  0  1  1  1 ]
[  1  0  0  0  0  0  0  0  0  0  0  1 -1 -1 ]
[  1  1  1  1  1  0  0  1  1  1  1  1 -1 -1 ]
[ -1 -1 -1 -1  1  0  0  1 -1 -1 -1 -1 -1 -1 ]
[ -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 ]


주어진 셀의 값과 다른 인접 셀들의 위치를 확인하고, 그 셀들이 동서남북 방향으로 시계방향 또는 반시계방향 순서대로 연속해서 있는지 확인하는 is_cascading_cell 함수



In [39]:

def is_cascading_cell(floorplan, cell):
    rows, cols = floorplan.shape
    room_value = floorplan[cell]
    
    # 방향: 북, 동, 남, 서
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    direction_names = ["북", "동", "남", "서"]
    
    # 셀이 유효한 범위에 있는지 확인하는 함수
    def is_valid(cell):
        return 0 <= cell[0] < rows and 0 <= cell[1] < cols
    
    # 인접 셀들 중 현재 셀의 값과 다른 셀들의 위치와 방향을 저장
    different_neighbors = []
    for idx, d in enumerate(directions):
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        if is_valid(neighbor) and floorplan[neighbor] != room_value:
            different_neighbors.append((neighbor, direction_names[idx]))
    
    # 인접 셀들의 방향이 시계방향 또는 반시계방향으로 연속되어 있는지 확인
    if len(different_neighbors) != 2:
        return False
    
    dir1 = direction_names.index(different_neighbors[0][1])
    dir2 = direction_names.index(different_neighbors[1][1])
    
    if (dir1 + 1) % 4 == dir2 or (dir1 - 1) % 4 == dir2:
        return True
    
    return False

In [41]:
# 테스트 실행
test_cells = [(0, 4), (0, 5), (0, 8),(0,9),(1,4),(1,7),(2,11),(2,12),(2,13),(3,9),(3,10), (4,8),(4,9),(5,2),(5,3),(5,6),(5,7),(4, 4), (0, 4), (0, 5)]
results = {cell: is_cascading_cell(floorplan, cell) for cell in test_cells}
print(results)

{(0, 4): True, (0, 5): False, (0, 8): True, (0, 9): False, (1, 4): True, (1, 7): True, (2, 11): False, (2, 12): True, (2, 13): False, (3, 9): True, (3, 10): True, (4, 8): True, (4, 9): True, (5, 2): False, (5, 3): False, (5, 6): True, (5, 7): True, (4, 4): False}


모든 셀에 대해 is_cascading_cell 함수를 적용하여 결과를 배열로 변환하는 방법

In [42]:
import numpy as np

def is_cascading_cell(floorplan, cell):
    rows, cols = floorplan.shape
    room_value = floorplan[cell]
    
    # 방향: 북, 동, 남, 서
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    direction_names = ["북", "동", "남", "서"]
    
    # 셀이 유효한 범위에 있는지 확인하는 함수
    def is_valid(cell):
        return 0 <= cell[0] < rows and 0 <= cell[1] < cols
    
    # 인접 셀들 중 현재 셀의 값과 다른 셀들의 위치와 방향을 저장
    different_neighbors = []
    for idx, d in enumerate(directions):
        neighbor = (cell[0] + d[0], cell[1] + d[1])
        if is_valid(neighbor) and floorplan[neighbor] != room_value:
            different_neighbors.append((neighbor, direction_names[idx]))
    
    # 인접 셀들의 방향이 시계방향 또는 반시계방향으로 연속되어 있는지 확인
    if len(different_neighbors) != 2:
        return False
    
    dir1 = direction_names.index(different_neighbors[0][1])
    dir2 = direction_names.index(different_neighbors[1][1])
    
    if (dir1 + 1) % 4 == dir2 or (dir1 - 1) % 4 == dir2:
        return True
    
    return False



In [43]:

# 코너 셀 여부를 저장할 배열 초기화
cascading_cells = np.zeros_like(floorplan, dtype=bool)

# 모든 셀에 대해 코너 셀 여부를 판단
for i in range(floorplan.shape[0]):
    for j in range(floorplan.shape[1]):
        cascading_cells[i, j] = is_cascading_cell(floorplan, (i, j))

# 결과 출력 (칸을 맞춰서)
for row in cascading_cells:
    print("[", " ".join(f"{int(x):2}" for x in row), "]")

[  0  0  0  0  1  0  0  0  1  0  0  0  0  0 ]
[  0  0  0  0  1  0  0  1  0  0  1  0  0  0 ]
[  0  0  0  0  0  0  0  0  0  0  0  0  1  0 ]
[  0  0  0  0  0  0  0  0  0  1  1  0  0  0 ]
[  0  1  0  0  0  0  0  1  1  1  0  0  1  0 ]
[  0  0  0  0  0  0  1  1  0  0  0  1  0  0 ]
[  0  0  0  1  0  1  1  0  1  0  0  0  0  0 ]
[  0  0  0  0  0  1  0  0  0  0  0  0  0  0 ]


이 cascading_cells 에서 1인 값을 랜덤하게 골라서 셀의 위치를 리턴


In [44]:
def random_cascading_cell(cascading_cells):
    indices = np.argwhere(cascading_cells == 1)
    if len(indices) == 0:
        return None
    return tuple(indices[np.random.choice(len(indices))])

# 결과 출력
for row in cascading_cells:
    print("[", " ".join(f"{int(x):2}" for x in row), "]")

# 랜덤하게 1인 값을 가진 셀의 위치를 반환
random_cell = random_cascading_cell(cascading_cells)
print("Random cascading cell:", random_cell)

[  0  0  0  0  1  0  0  0  1  0  0  0  0  0 ]
[  0  0  0  0  1  0  0  1  0  0  1  0  0  0 ]
[  0  0  0  0  0  0  0  0  0  0  0  0  1  0 ]
[  0  0  0  0  0  0  0  0  0  1  1  0  0  0 ]
[  0  1  0  0  0  0  0  1  1  1  0  0  1  0 ]
[  0  0  0  0  0  0  1  1  0  0  0  1  0  0 ]
[  0  0  0  1  0  1  1  0  1  0  0  0  0  0 ]
[  0  0  0  0  0  1  0  0  0  0  0  0  0  0 ]
Random cascading cell: (6, 8)



- `is_valid` 함수:
주어진 셀이 유효한 범위 내에 있는지 확인합니다.
- `boundary_cell_room` 함수:
floorplan 배열과 같은 크기의 boundary_cells 배열을 생성합니다. floorplan의 값이 -1인 셀은 그대로 -1로 설정하고, 나머지 셀은 0으로 초기화합니다.
각 셀의 값을 확인하고, 인접 셀 중 하나라도 값이 다르거나 경계를 벗어나는 경우 해당 셀을 경계 셀로 간주하여 1로 설정합니다.
- 경계 셀 여부를 저장할 배열 계산 및 출력:
이중 for 루프를 사용하여 floorplan의 모든 셀을 순회하며 boundary_cell_room 함수를 적용하고, 그 결과를 boundary_cells 배열에 저장합니다.
각 행을 출력할 때 값을 일정한 폭으로 맞춰서 출력합니다.
- 결과
위 코드를 실행하면 boundary_cells 배열이 출력됩니다. 이 배열은 floorplan의 각 셀에 대해 경계 셀인지 여부를 1/0으로 나타내며, 원래 -1인 셀은 그대로 -1로 유지됩니다.

In [50]:
import numpy as np

def boundary_cell_room(floorplan):
    rows, cols = floorplan.shape

    # 방향: 북, 동, 남, 서
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    
    # 셀이 유효한 범위에 있는지 확인하는 함수
    def is_valid(cell):
        return 0 <= cell[0] < rows and 0 <= cell[1] < cols

    # 결과 배열 초기화 (-1은 그대로 유지)
    boundary_cells = np.where(floorplan == -1, -1, 0)

    for i in range(rows):
        for j in range(cols):
            if floorplan[i, j] == -1:
                continue
            current_value = floorplan[i, j]
            for d in directions:
                neighbor = (i + d[0], j + d[1])
                # 경계 셀은 배열의 경계를 벗어나는 셀과 인접한 셀로 정의
                if not is_valid(neighbor) or floorplan[neighbor] != current_value:
                    boundary_cells[i, j] = 1
                    break

    return boundary_cells

# 테스트 배열
floorplan = np.array([
    [5, 4, 4, 4, 4, 3, 3, 3, 3, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, -1, -1, -1, -1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1],
    [5, 4, 4, 4, 3, 3, 3, 2, 2, 1, 1, 1, -1, -1],
    [5, 5, 5, 4, 3, 3, 3, 1, 1, 1, 1, 1, -1, -1],
    [-1, -1, -1, -1, 3, 3, 1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 3, 1, 1, 1, -1, -1, -1, -1, -1, -1]
])

# 경계 셀 여부를 저장할 배열 계산
boundary_cells = boundary_cell_room(floorplan)

# 결과 출력 (칸을 맞춰서)
for row in boundary_cells:
    print("[", " ".join(f"{x:2}" for x in row), "]")


[  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 ]
[  1  1  0  1  1  0  1  1  1  1 -1 -1 -1 -1 ]
[  1  1  0  1  1  0  1  1  0  0  1  1  1  1 ]
[  1  1  0  1  1  0  1  1  0  1  1  1  1  1 ]
[  1  1  1  1  1  0  1  1  1  1  0  1 -1 -1 ]
[  1  1  1  1  1  0  1  1  1  1  1  1 -1 -1 ]
[ -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 ]
[ -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 ]
